## Part 2

Getting a DataFrame for the name of the countries

In [1]:
import pandas as pd
import nltk
from nltk.book import FreqDist
from nltk.corpus import stopwords
import os
from os import path
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from helpers import *
%matplotlib inline

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


In [251]:
#Let's get a DataFrame with the different countries in the world, so that we can get the number of mentions of them
#within the emails.
#First let's take a look in the structure of the pycountry object.
list(pycountry.countries)[0]

Country(alpha_2='AW', alpha_3='ABW', name='Aruba', numeric='533')

In [458]:
#We can see short forms for the name of the country as well as the name. Let's get the DataFrame with those values:
countries=[]
for i in range(len(list(pycountry.countries))):
    countries.append([list(pycountry.countries)[i].name,list(pycountry.countries)[i].name.lower(),list(pycountry.countries)[i].alpha_2, list(pycountry.countries)[i].alpha_3])
countries_df=pd.DataFrame(countries,columns=['name','name2','alpha2','alpha3'])
countries_df.head()

,name,name2,alpha2,alpha3
0,Aruba,aruba,AW,ABW
1,Afghanistan,afghanistan,AF,AFG
2,Angola,angola,AO,AGO
3,Anguilla,anguilla,AI,AIA
4,Åland Islands,åland islands,AX,ALA


Getting the texts of the emails

In [254]:
#First let's read the csv and get a DataFrame of the emails
emails_df= pd.read_csv('hillary-clinton-emails/Emails.csv')

In [255]:
emails_df.head()

,Id,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,...,ExtractedTo,ExtractedFrom,ExtractedCc,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText
0,1,C05739545,WOW,H,"Sullivan, Jacob J",87.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739545...,F-2015-04841,...,NaN,"Sullivan, Jacob J <Sullivan11@state.gov>",NaN,"Wednesday, September 12, 2012 10:16 AM",F-2015-04841,C05739545,05/13/2015,RELEASE IN FULL,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
1,2,C05739546,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,H,NaN,NaN,2011-03-03T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739546...,F-2015-04841,...,NaN,NaN,NaN,NaN,F-2015-04841,C05739546,05/13/2015,RELEASE IN PART,"B6\nThursday, March 3, 2011 9:45 PM\nH: Latest...",UNCLASSIFIED\nU.S. Department of State\nCase N...
2,3,C05739547,CHRIS STEVENS,;H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739547...,F-2015-04841,...,B6,"Mills, Cheryl D <MillsCD@state.gov>","Abedin, Huma","Wednesday, September 12, 2012 11:52 AM",F-2015-04841,C05739547,05/14/2015,RELEASE IN PART,Thx,UNCLASSIFIED\nU.S. Department of State\nCase N...
3,4,C05739550,CAIRO CONDEMNATION - FINAL,H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739550...,F-2015-04841,...,NaN,"Mills, Cheryl D <MillsCD@state.gov>","Mitchell, Andrew B","Wednesday, September 12,2012 12:44 PM",F-2015-04841,C05739550,05/13/2015,RELEASE IN PART,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
4,5,C05739554,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,"Abedin, Huma",H,80.0,2011-03-11T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739554...,F-2015-04841,...,NaN,NaN,NaN,NaN,F-2015-04841,C05739554,05/13/2015,RELEASE IN PART,"H <hrod17@clintonemail.com>\nFriday, March 11,...",B6\nUNCLASSIFIED\nU.S. Department of State\nCa...


We want to create a new dataframe with the concatenation of the email's subjectand the email's body so that we can make the sentiment analisys.

In [140]:
emails_df.columns

Index(['Id', 'DocNumber', 'MetadataSubject', 'MetadataTo', 'MetadataFrom',
       'SenderPersonId', 'MetadataDateSent', 'MetadataDateReleased',
       'MetadataPdfLink', 'MetadataCaseNumber', 'MetadataDocumentClass',
       'ExtractedSubject', 'ExtractedTo', 'ExtractedFrom', 'ExtractedCc',
       'ExtractedDateSent', 'ExtractedCaseNumber', 'ExtractedDocNumber',
       'ExtractedDateReleased', 'ExtractedReleaseInPartOrFull',
       'ExtractedBodyText', 'RawText'],
      dtype='object')

In [148]:
#Let's take a look at the 'ExtractedSubject' and 'ExtractedBodyText' columns
emails_df.ExtractedSubject.head()

0                            FW: Wow
1                                NaN
2                  Re: Chris Stevens
3    FVV: Cairo Condemnation - Final
4                                NaN
Name: ExtractedSubject, dtype: object

In [164]:
emails_df.ExtractedBodyText.head()

0                                                  NaN
1    B6\nThursday, March 3, 2011 9:45 PM\nH: Latest...
2                                                  Thx
3                                                  NaN
4    H <hrod17@clintonemail.com>\nFriday, March 11,...
Name: ExtractedBodyText, dtype: object

In [514]:
#We can see that there are NaN in the columns, so first we need to see if for the same row, both of the values
#for both columns are NaN so we can discard that row.
print("Number of NaN in ExtractedBodyText column: %d" % sum(emails_df.ExtractedBodyText.isnull()))
print("Number of NaN in ExtractedSubject column: %d" % sum(emails_df.ExtractedSubject.isnull()))
emails_test=emails_df[~(emails_df.ExtractedBodyText.isnull() & emails_df.ExtractedSubject.isnull())]
#Because we want to concatenate both columns we are going to replace the NaN by a blank space
#We also saw that there are new lines "\n" so we also want to replace them by a blank space so that we can separate
#the words between new lines
emails_test=emails_test.fillna(value=" ").replace(to_replace="\n",value=" ",regex=True)

Number of NaN in ExtractedBodyText column: 1203
Number of NaN in ExtractedSubject column: 1685


In [515]:
#After getting rid of the rows with NaN in both columns mentioned above, we can concatenate them
email_texts=emails_test['ExtractedSubject'].str.cat(emails_test['ExtractedBodyText'], sep=' ')
email_texts=email_texts.to_frame();

In [516]:
email_texts['tokens']=email_texts.ExtractedSubject.apply(word_tokenize)
email_texts.columns = ['text', 'tokens']

In [520]:
countries_df.head()

,name,name2,alpha2,alpha3
0,Aruba,aruba,AW,ABW
1,Afghanistan,afghanistan,AF,AFG
2,Angola,angola,AO,AGO
3,Anguilla,anguilla,AI,AIA
4,Åland Islands,åland islands,AX,ALA


In [521]:
email_texts.head()

,text,tokens
0,FW: Wow,"[FW, :, Wow]"
1,"B6 Thursday, March 3, 2011 9:45 PM H: Latest...","[B6, Thursday, ,, March, 3, ,, 2011, 9:45, PM,..."
2,Re: Chris Stevens Thx,"[Re, :, Chris, Stevens, Thx]"
3,FVV: Cairo Condemnation - Final,"[FVV, :, Cairo, Condemnation, -, Final]"
4,"H <hrod17@clintonemail.com> Friday, March 11...","[H, <, hrod17, @, clintonemail.com, >, Friday,..."


In [524]:
sia = SentimentIntensityAnalyzer()
sentiment=[]
for sentence in email_texts['text']:
        ss = sid.polarity_scores(sentence)['compound']
        sentiment.append(ss)
        

In [ ]:
email_texts['sentiment']=sentiment

In [536]:
email_texts

,text,tokens,sentiment
0,FW: Wow,"[FW, :, Wow]",0.5859
1,"B6 Thursday, March 3, 2011 9:45 PM H: Latest...","[B6, Thursday, ,, March, 3, ,, 2011, 9:45, PM,...",0.0000
2,Re: Chris Stevens Thx,"[Re, :, Chris, Stevens, Thx]",0.3612
3,FVV: Cairo Condemnation - Final,"[FVV, :, Cairo, Condemnation, -, Final]",-0.5859
4,"H <hrod17@clintonemail.com> Friday, March 11...","[H, <, hrod17, @, clintonemail.com, >, Friday,...",0.0000
5,Meet The Right Wing Extremist Behind Anti-Musl...,"[Meet, The, Right, Wing, Extremist, Behind, An...",-0.9062
6,"FW: Anti-Muslim film director in hiding, follo...","[FW, :, Anti-Muslim, film, director, in, hidin...",-0.7430
7,"H <hrod17@clintonemail.corn> Friday, March 1...","[H, <, hrod17, @, clintonemail.corn, >, Friday...",0.0000
8,FVV: Secretary's remarks FYI,"[FVV, :, Secretary, 's, remarks, FYI]",0.3680
9,"more on Libya B6 Wednesday, September 12, 2012...","[more, on, Libya, B6, Wednesday, ,, September,...",0.0000
